In [4]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import random
from glob import glob
import fnmatch
import collections
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers, initializers, optimizers, constraints
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.python.keras.utils import conv_utils
import numpy as np




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/EURO SAT/EuroSat_Custom/Train'
val_path ='/content/drive/MyDrive/EURO SAT/EuroSat_Custom/Val'
test_path ='/content/drive/MyDrive/EURO SAT/EuroSat_Custom/Test'
batch_size = 64
img_height = 64
img_width = 64
no_of_classes = 10
classes_name = [0,1,2,3,4,5,6,7,8,9]
input_shape=(img_height,img_width,3)

tf.random.set_seed(1142)
random_seed= np.random.seed(1142)

datagen = ImageDataGenerator(
    rescale = 1. / 255,
    featurewise_center=False,
    horizontal_flip=True,
    vertical_flip=False,
    featurewise_std_normalization=True)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    val_path,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)

print("Train generator")
print(train_generator.class_indices)
print("Validation generator")
print(validation_generator.class_indices)

Found 9080 images belonging to 10 classes.
Found 1599 images belonging to 10 classes.
Train generator
{'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}
Validation generator
{'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}


In [ ]:
print(train_generator[1])

(array([[[[1.        , 0.9803922 , 0.95294124],
         [0.9294118 , 0.854902  , 0.8313726 ],
         [0.8313726 , 0.7568628 , 0.73333335],
         ...,
         [1.        , 0.882353  , 0.86274517],
         [0.91372555, 0.80392164, 0.7607844 ],
         [0.91372555, 0.80392164, 0.7568628 ]],

        [[1.        , 0.97647065, 0.95294124],
         [0.9333334 , 0.85098046, 0.8313726 ],
         [0.8431373 , 0.75294125, 0.7294118 ],
         ...,
         [0.9960785 , 0.882353  , 0.86666673],
         [0.91372555, 0.80392164, 0.7607844 ],
         [0.91372555, 0.80392164, 0.7568628 ]],

        [[0.8235295 , 0.7254902 , 0.70980394],
         [0.86666673, 0.7686275 , 0.7490196 ],
         [0.9176471 , 0.8117648 , 0.7843138 ],
         ...,
         [0.8588236 , 0.77647066, 0.76470596],
         [0.8862746 , 0.7803922 , 0.74509805],
         [0.89019614, 0.7803922 , 0.7372549 ]],

        ...,

        [[0.9568628 , 0.8941177 , 0.8431373 ],
         [1.        , 0.95294124, 0.91372555

In [ ]:
def VGG16():

  input = tf.keras.layers.Input(shape=(64,64,3))


  conv1= tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(input)
  conv2= tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv1)
  pool1=tf.keras.layers.MaxPooling2D((2,2),(2,2))(conv2)
  conv3= tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(pool1)
  conv4= tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv3)
  pool2=tf.keras.layers.MaxPooling2D((2,2),(2,2))(conv4)
  conv5= tf.keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(pool2)
  conv6= tf.keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv5)
  conv7= tf.keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv6)
  pool3=tf.keras.layers.MaxPooling2D((2,2),(2,2))(conv7)
  conv8= tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(pool3)
  conv9= tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv8)
  conv10= tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv9)
  pool4=tf.keras.layers.MaxPooling2D((2,2),(2,2))(conv10)
  conv11= tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(pool4)
  conv12= tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv11)
  conv13= tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding ='same',kernel_initializer='he_normal')(conv12)
  pool5=tf.keras.layers.MaxPooling2D((2,2),(2,2))(conv13)

  global_pool=tf.keras.layers.GlobalAveragePooling2D()(pool5)

  dense1=tf.keras.layers.Dense(2048,activation ='relu')(global_pool)
  dense2=tf.keras.layers.Dense(2048,activation ='relu')(dense1)
  output=tf.keras.layers.Dense(10,activation='softmax')(dense2)

  model = tf.keras.models.Model(inputs=input,outputs=output)
  return model

model=VGG16()
model.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =["acc"])
model.summary()





Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 64)        1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 32, 32, 128)       147584    
                                                           

In [ ]:

!ls '/content/drive/MyDrive/EURO SAT/EuroSat_Custom/Train'


history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch= len(train_generator),
                    validation_steps= len(validation_generator),
                    epochs=10
        )

model.save('/content/drive/MyDrive/EURO SAT/Euro_VGG16.h5')

AnnualCrop  HerbaceousVegetation  Industrial  PermanentCrop  River
Forest	    Highway		  Pasture     Residential    SeaLake
Epoch 1/10
142/142 [==============================] - 2410s 17s/step - loss: 3.0916 - acc: 0.2331 - val_loss: 1.7273 - val_acc: 0.2933
Epoch 2/10
142/142 [==============================] - 1927s 14s/step - loss: 1.4741 - acc: 0.4087 - val_loss: 1.5415 - val_acc: 0.4009
Epoch 3/10
142/142 [==============================] - 1902s 13s/step - loss: 1.3458 - acc: 0.4716 - val_loss: 1.2732 - val_acc: 0.4909
Epoch 4/10
142/142 [==============================] - 1944s 14s/step - loss: 1.2125 - acc: 0.5416 - val_loss: 1.1939 - val_acc: 0.5478
Epoch 5/10
142/142 [==============================] - 1966s 14s/step - loss: 1.1441 - acc: 0.5689 - val_loss: 1.1057 - val_acc: 0.5697
Epoch 6/10
142/142 [==============================] - 1901s 13s/step - loss: 1.1342 - acc: 0.5634 - val_loss: 1.0431 - val_acc: 0.5972
Epoch 7/10
142/142 [==============================] - 1952s 14s/st

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
model2=load_model('/content/drive/MyDrive/EURO SAT/Euro_VGG16.h5')

In [7]:
test_d = ImageDataGenerator(rescale=1. / 255)
test = test_d.flow_from_directory('/content/drive/MyDrive/EURO SAT/EuroSat_Custom/Test',
                                  target_size=(64, 64),
                                  batch_size=4,
                                  shuffle=False,
                                  class_mode='categorical')

Found 1440 images belonging to 10 classes.


In [14]:
# Making predictions on the test data
test_step = len(test)
test.reset()
pred = model2.predict(test, steps=test_step, verbose=1)
pred_class_indices = np.argmax(pred, axis=1)

# Printing predicted labels
print(pred_class_indices)

360/360 ━━━━━━━━━━━━━━━━━━━━ 287s 799ms/step
[0 0 0 ... 9 9 9]


In [15]:
# Import necessary libraries
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, roc_auc_score, auc, f1_score, jaccard_score, classification_report
from sklearn.metrics import precision_score, recall_score, jaccard_score

# Define the classes
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Evaluate the model on the test data and calculate metrics for each class
for cl in classes:
    print("class: ", cl)
    a1 = np.uint8(test.labels == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print("precision {}".format(precision_score(y_true=a1, y_pred=a2)))
    print("recall {}".format(recall_score(y_true=a1, y_pred=a2)))
    print("F1 {}".format(f1_score(y_true=a1, y_pred=a2)))
    print("jaccard {}".format(jaccard_score(y_true=a1, y_pred=a2)))
    print("Accuracy {}".format(accuracy_score(y_true=a1, y_pred=a2)))
    print("\n------------------------\n")

class:  0
precision 0.7142857142857143
recall 0.7986111111111112
F1 0.7540983606557378
jaccard 0.6052631578947368
Accuracy 0.9479166666666666

------------------------

class:  1
precision 0.8
recall 0.6944444444444444
F1 0.7434944237918216
jaccard 0.591715976331361
Accuracy 0.9520833333333333

------------------------

class:  2
precision 0.34408602150537637
recall 0.2222222222222222
F1 0.270042194092827
jaccard 0.15609756097560976
Accuracy 0.8798611111111111

------------------------

class:  3
precision 0.7168141592920354
recall 0.5625
F1 0.6303501945525292
jaccard 0.4602272727272727
Accuracy 0.9340277777777778

------------------------

class:  4
precision 0.8979591836734694
recall 0.9166666666666666
F1 0.9072164948453607
jaccard 0.8301886792452831
Accuracy 0.98125

------------------------

class:  5
precision 0.4980237154150198
recall 0.875
F1 0.6347607052896725
jaccard 0.46494464944649444
Accuracy 0.8993055555555556

------------------------

class:  6
precision 0.50980392156862

In [16]:
# Print overall performance metrics
print("Accuracy {}".format(accuracy_score(y_true=test.labels, y_pred=pred_class_indices)))
print("F1 {}".format(f1_score(y_true=test.labels, y_pred=pred_class_indices, average="macro")))
print("precision {}".format(precision_score(y_true=test.labels, y_pred=pred_class_indices, average="macro")))
print("recall {}".format(recall_score(y_true=test.labels, y_pred=pred_class_indices, average="macro")))
print("jaccard {}".format(jaccard_score(y_true=test.labels, y_pred=pred_class_indices, average="macro")))
print("classification report\n {}".format(classification_report(y_true=test.labels, y_pred=pred_class_indices)))
print('\n\n')

Accuracy 0.6805555555555556
F1 0.6677603237814889
precision 0.6756227896686731
recall 0.6805555555555555
jaccard 0.5301458393781232
classification report
               precision    recall  f1-score   support

           0       0.71      0.80      0.75       144
           1       0.80      0.69      0.74       144
           2       0.34      0.22      0.27       144
           3       0.72      0.56      0.63       144
           4       0.90      0.92      0.91       144
           5       0.50      0.88      0.63       144
           6       0.51      0.36      0.42       144
           7       0.80      0.92      0.85       144
           8       0.66      0.56      0.61       144
           9       0.81      0.90      0.85       144

    accuracy                           0.68      1440
   macro avg       0.68      0.68      0.67      1440
weighted avg       0.68      0.68      0.67      1440




